<a href="https://colab.research.google.com/github/halim-jun/Ai_study/blob/master/20200812_%ED%85%8D%EC%8A%A4%ED%8A%B8%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. 크롤러 제작

In [24]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
!pip install newspaper3k
from newspaper import Article
!pip install requests


ModuleNotFoundError: ignored

In [32]:
!pip install konlpy
from konlpy.tag import Komoran
tokenizer = Komoran()

In [10]:
def make_urllist(page_num, code, date):
  urllist=[]
  for i in range(1,page_num+1):
    url = 'https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1='+str(code)+'&date='+str(date)+'&page='+str(i)   
    news=requests.get(url)
    soup=BeautifulSoup(news.content, 'html.parser')

    news_list=soup.select('.newsflash_body .type06_headline li dl')
    news_list.extend(soup.select('.newsflash_body .type06 li dl'))
    
    for line in news_list:
      urllist.append(line.a.get('href'))
  return urllist

In [11]:
idx2word = {'101' : '경제', '102' : '사회', '103' : '생활/문화', '105' : 'IT/과학'}
idx2word.keys()

dict_keys(['101', '102', '103', '105'])

In [12]:
def make_data(urllist, code):
  text_list = []
  for url in urllist:
    article = Article(url, language='ko')
    article.download()
    article.parse()
    text_list.append(article.text)

  df = pd.DataFrame({'news': text_list})

  df['code'] = idx2word[str(code)]
  return df

In [13]:
def make_total_data(page_num, code_list, date):
  df = None

  for code in code_list:
    url_list = make_urllist(page_num, code, date)
    df_temp = make_data(url_list, code)

    if df is not None:
      df = pd.concat([df, df_temp])
    else:
      df = df_temp

  return df

#2. 크롤링 진행

- 정확도를 높이기 위해서 날짜를 다양화 => 

In [52]:
df=None

date_list=[20200301,20200310,20200401,20200410,20200501,202005010,20200520,20200601, 20200610,20200630, 20200701, 20200801]
for date in date_list:
  return_data=make_total_data(1,[101,102,103,105],date)
  if df is not None:
    df=pd.concat([df, return_data])
  else:
    df=return_data

In [68]:
from google.colab import files
df.to_csv('20200804_전하림.csv') 
files.download('20200804_전하림.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [54]:
df['news'] = df['news'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
df.drop_duplicates(subset=['news'], inplace=True)
df

,news,code
0,동영상 뉴스코로나로 위축되는 내수를 살리기 위해 정부가 개별소비세 인하를 모든 승용...,경제
1,동영상 뉴스앵커인터넷에서 해외 직구로 쉽게 살 수 있는 초저가 어린이 휴대용 카시트...,경제
2,코로나가 세계적으로 점점 확산되면서 종합상사를 비롯한 국내 기업들의 해외영업에 비상...,경제
3,육군 마스크 생산업체 지원나섰다 일 인천의 한 코로나 예방용 마스크 생산업체에서 육...,경제
4,발길 끊긴 대구 동성로 일 오후 대구 동성로의 한 상점에 코로나 확산을 막기 위해 ...,경제
...,...,...
15,이데일리 김현아 기자온라인 동영상 서비스 웨이브 대표 이태현가 오늘 유니버설로부터 ...,IT/과학
16,이데일리 김현아 기자대한민국 대표 중고거래 플랫폼 중고나라대표이승우가 이용자의 안전...,IT/과학
17,바라카원전 사진한국전력지디넷코리아주문정 기자아랍에미리트연합 원전 주계약자인 한국전...,IT/과학
18,이데일리 김현아 기자가상자산암호화폐거래소 지닥이 월 일부터 카카오 자회사의 발행 가...,IT/과학


In [55]:
df['code'].value_counts()

IT/과학    207
생활/문화    197
경제       196
사회       186
Name: code, dtype: int64

#3. Token화 
- Stop words 제외
- **데이터 처리 속도가 상대적으로 빠르다고 알려졌기 때문에 Komoran 사용**


In [71]:
# 토큰화 및 토큰화 과정에서 불용어를 제거하는 함수입니다.
stopwords = ['에','것', 'ㄴ','는','은','을','했','에게','있','이','의','하','한','다','과','때문','할','수','무단','따른','및','금지','전재','경향신문','기자','습니다','는데','가','앵커','등','들','파이낸셜','저작','등','뉴스','노컷뉴스', '저작권자','ㄹ','네','습니다','죠','아','었','를','라는','었','에서','와','후','아','고','었','아',
             '어서','되']

def preprocessing(data):
  text_data = []

  for sentence in data:
    temp_data = []
    #- 토큰화
    temp_data = tokenizer.morphs(sentence) 
    #- 불용어 제거
    temp_data = [word for word in temp_data if not word in stopwords] 
    text_data.append(temp_data)

  text_data = list(map(' '.join, text_data))

  return text_data


In [72]:
text_data = preprocessing(df['news'])
len(stopwords)

55

In [73]:
text_data[1:5]

['동영상 인터넷 해외 직구 로 쉽 게 살 초 저가 어린이 휴대 용 카 시트 안전 지 않 다는 소비자 원 조사 결과 나오 았 소비자 원 문제 제품 안전 벨트 아예 착용 지 않 다르 바 없 수준 라고 지적 았 차 유정 보도 ㅂ니다 인터넷 검색 만 면 나오 어린이 휴대 용 카 시트 ㅂ니다 주로 천 재질 ㄴ데 제품 좌석 고정 뒤 벨트 채우 중간 패드 좌석 안전띠 통과 시키 어 이중 으로 고정 방식 입 니다 해외 직구 로 살 가격 도 만 원 미만 으로 저렴 ㅂ니다 그러 ㄴ데 문제 안전 입 니다 소비자 원 저가 형 카 시트 개 제품 살피 더니 인증 받 제품 단 개 도 없 기능 실험 결과 도 충격 적 소비자 원 미 인증 시트 개 고르 장착 차량 시속 속도 로 벽 부딪히 게 보 았 문제 제품 보 면 목 부위 심하 게 꺾이 가슴 골반 부위 시트 완전히 떨어지 ㅂ니다 인증 제품 마네킹 신체 전반 안정 적 으로 고정 상체 앞 으로 내 로 쏠리 반면 미 인증 품 까 지 튀 어 나가 ㅂ니다 또 다른 미 인증 제품 도 마 찬가 집 니다 고정 장치 망가지 면서 마네킹 하체 시트 앞 으로 그대로 미끄러지 ㅂ니다 사 충격 실험 뒤 미 인증 제품 입 니다 골반 고정 장치 완전히 떨어지 어 나가 았 정도 면 카 시트 로 보호 기능 거의 없 다는 게 소비자 원 판단 입 니다 김 병법 소비자 원 안전 감시 국 생활 안전 팀 저희 느끼 던 안전 벨트 착용 지 않 크 차이 없 다는 겁 니다 일부 미 인증 제품 피부염 이나 호흡기 질환 유발 유해 물질 폼알데하이드 도 기준 치 초과 검출 기 도 았 소비자 원 지금 문제 제품 팔 지 말 도록 조치 았 다면서 소비자 도 안전 인증 표시 없 저가 형 제품 사지 말 라고 당부 았 차 유정 입 니다 저작권 자 재 배포',
 '코로나 세계 적 으로 점점 확산 면서 종합상사 비롯 국내 기업 해외 영업 비상 걸리 사태 초기 발병 국인 중국 출장 자제 았 지만 최근 확진 자 급증 한국 해외 출장 기피 대상 국인 입국 제한 국가 속출 데 입국 더라도 격리 조치 당하 해

#3. 머신러닝

In [74]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

In [83]:
X_train, X_test, y_train, y_test = train_test_split(text_data, df['code'], random_state = 10)

In [80]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [81]:
def tfidf_vectorizer(data):
  data_counts = count_vect.transform(data)
  data_tfidf = tfidf_transformer.transform(data_counts)
  return data_tfidf

In [84]:
y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       IT/과학       0.83      0.88      0.85        50
          경제       0.80      0.80      0.80        46
          사회       0.83      0.88      0.85        49
       생활/문화       0.93      0.83      0.88        52

    accuracy                           0.85       197
   macro avg       0.85      0.85      0.85       197
weighted avg       0.85      0.85      0.85       197

